In [ ]:
# This is an example of pulling data from the Mountain Project API (with Nick's PERSONAL API key).
# Your specific request URL will vary, but the command should be similar

response4 = requests.get("https://www.mountainproject.com/data/get-routes-for-lat-lon?lat=34.02&lon=-116.16&maxDistance=50&maxResults=500&minDiff=V3&maxDiff=V17&key=200588892-9fbd8573f0fe567a7983d3ab5ae62924")
print(response4)

In [ ]:
# This allows you to see the structure of the JSON data, not in a string

import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent = 4)
    print(text)
    
jprint(response4.json())

In [ ]:
# Here, I jumped forward one level up the tree in the JSON file to make it easier to subset data components.

routes4 = response4.json()['routes']
jprint(routes4)

In [ ]:
# I wanted to check the latitude and longitude of each of the routes from this one request, so I wrote out loops
# to print out that information. Also used to generate the x and y values for a plot.

latV7 = [] # finding the range of latitudes

for i in routes4:
    lati = i['latitude']
    latV7.append(lati)
    
print(latV7)
print()

longV7 = [] # finding the range of longitudes

for i in routes4:
    longi = i['longitude']
    longV7.append(longi)
    
print(longV7)

In [ ]:
# Plotting out all of the routes from this first request

import numpy as np
import matplotlib.pyplot as plt

N = len(latV7)
#x = long
#y = lat
#xv3 = longV3
#yv3 = latV3
xv7 = longV7
yv7 = latV7

#plt.scatter(x, y, color="green",label="V1")
#plt.scatter(xv3, yv3, color="orange",label="V3")
plt.scatter(xv7, yv7, color="red",label="V7")
plt.xlabel("Longitude Coordinates")
plt.ylabel("Latitude Coorindates")
plt.title("Bouldering Routes Near Outback Bouldering Area")
plt.xticks(rotation=45)
plt.legend(loc='lower left')
plt.show()

In [ ]:
# FOR PREPARING TO SEND TO A SQL DATABASE
# First step - take the json file and use the .dumps command to prepare to put the data into a data frame.

routez = response4.json()['routes']
#jprint(routes)
json.dumps(routez)

In [ ]:
# Defining the data frame

df1=DataFrame(routez)
df1

In [4]:
## At this point, make sure that PostgreSQL is RUNNING

# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'jtnp_boulders' # change this to whatever you want your database to be called
username = 'nicholasgriffin' # change this to your username

In [5]:
# Establishing the ENGINE - getting this notebook to talk to the SQL database you're working with.

## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

NameError: name 'create_engine' is not defined

In [ ]:
## create a database (if it doesn't exist)
## If this is your first time defining a new SQL database, then you will need to run these lines to make
## you new database actually exist.

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

In [ ]:
# Sending the data frame defined above to SQL.
# Within the parentheses: (name of data within the database, engine name[in this case, 'engine'], if_exists)
# The if_exists command has three options: 
# 'replace' means overwriting any data already in your chunk of saved data. For a brand new database, this option works.
# 'append' means adding additional data to what you have already saved, good for multiple API requests and adding more info to a database
# 'fail' means Raise a ValueError (THIS IS THE DEFAULT OPTION - YOU SHOULD CHOOSE 'REPLACE' OR 'APPEND')


df1.to_sql('jtnp_routes_v1', engine, if_exists='replace')

In [ ]:
# After the data is saved in your database, you need to run this to be able to view that information.

# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [3]:
# Running a SQL query from your saved data/database
# query:

sql_query = """
SELECT * FROM jtnp_routes_v1 WHERE rating='V6';
"""

# SELECT * FROM jtnp_routes_v1 WHERE rating='V6';
# """

jtnp_from_sql = pd.read_sql_query(sql_query,con)
jtnp_from_sql.head()

NameError: name 'pd' is not defined